# Derive Top N Portfolio

In [78]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [113]:
import os
import sys
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
# import rezaware as reza
# from utils.modules.etl.load import sparkDBwls as sdb
# from utils.modules.etl.transform import sparkCleanNRich as scne
from mining.modules.assets.etp import dailyTopN as topN
# from utils.modules.ml.timeseries import rollingstats as stats

''' restart initiate classes '''
if debug:
    import importlib
    topN = importlib.reload(topN)
    
__desc__ = "analyze crypto market capitalization time series data"
clsMPT =topN.WeightedPortfolio(desc=__desc__)

print("\nClass initialization and load complete!")

All functional DAILYTOPN-libraries in ETP-package of ASSETS-module imported successfully!
sparkNoSQLwls Class initialization complete
sparkNoSQLwls Class initialization complete
execSession Class initialization complete
performIndex Class initialization complete
execSession Class initialization complete
dailyTopN Class initialization complete

Class initialization and load complete!


## Build the Weighted Portfolio
We apply a query to select assets with mcap > 1.0 million. Any missing values are imputed with the mean value.

In [117]:
_date = date(2022,1,27)
_portfolio = clsMPT.build_weighted_portfolio(
    mcap_date=_date,
    pick_asset_db_name=None,
    mcap_ror_db_name=None,
)

In [118]:
from pyspark.sql import functions as F

clsMPT.data.select(
    F.col('mcap_past_pk'),F.col('mcap_date'),
    F.col('asset_name'),F.col('mcap_value'),F.col('log_ror'))\
    .sort(F.col('mcap_date'))\
    .show(n=10)

print("weighted portfolio\n",_portfolio)

+------------+-------------------+----------+--------------------+------------------+
|mcap_past_pk|          mcap_date|asset_name|          mcap_value|           log_ror|
+------------+-------------------+----------+--------------------+------------------+
|      194922|2022-01-27 00:00:00|      near|7946585687.461500...|0.7168222024097004|
|      166625|2022-01-27 00:00:00|       axs|3958484505.296750...|0.7054650037199550|
|       19803|2022-01-27 00:00:00|       omi|4405628167.189120...|1.2073031525548736|
|       92278|2022-01-27 00:00:00|      hbar|4228933531.966320...|0.6999991520948953|
|      113027|2022-01-27 00:00:00|       leo|3390804647.234960...|0.6946924859602940|
|       95169|2022-01-27 00:00:00|      klay|3316715769.769650...|0.6582358088427600|
|      210413|2022-01-27 00:00:00|       icp|4335482138.021870...|0.6940572663062533|
|         380|2022-01-27 00:00:00|      wbtc|10399154950.87140...|0.6949849907517710|
|       53843|2022-01-27 00:00:00|     matic|122129157

## Write MPT to MongoDB

* Collection name = "mpt."+date(YYYY-MM-DD)
* document structure: \_id, date, asset, mcap.value, mcap.weight, mcap.ror

In [119]:
_kwargs = {
    "DESTINDBNAME":'tip-daily-mpt',
    "COLLPREFIX" : 'mpt'
}
mpt_list_ = clsMPT.write_mpt_to_db(
    portfolio_data=_portfolio,
    cols_dict=_cols,
    **_kwargs,
)
print("Upsert %d documents" % len(mpt_list_))

Upsert 1 documents


In [6]:
''' DEPRECATED moved to dailyTopN build_weighted_portfolio function'''
_from_date = '2022-01-25'
_to_date = '2022-01-25'
_db_name = "warehouse.mcap_past"
_kwargs={}
# _query = "select * from warehouse.mcap_past "+\
#         f"where mcap_date between '{_from_date}' and '{_to_date}' "+\
#         f"and mcap_value > 1000000 and asset_name in "+\
#         "('brg_x','_crdn','avme','atri','ethix','hoge','xpx','wabi','dmg','mintme','chart')"
_query =f"select * from {_db_name} "+\
        f"where mcap_date between '{_from_date}' and '{_to_date}' "+\
        f"and mcap_value > 1000000 and log_ror is not null"

mcap_sdf = clsMPT.read_ror(select=_query,**_kwargs)

print("Loaded %d rows and %d columns" % (mcap_sdf.count(),len(mcap_sdf.columns)))

23/04/04 10:19:37 WARN Utils: Your hostname, FarmRaiderTester resolves to a loopback address: 127.0.1.1; using 192.168.124.15 instead (on interface enp2s0)
23/04/04 10:19:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/04/04 10:19:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/04 10:19:39 WARN FileSystem: Cannot load filesystem: java.util.ServiceConfigurationError: org.apache.hadoop.fs.FileSystem: com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem Unable to get public no-arg constructor
23/04/04 10:19:39 WARN FileSystem: java.lang.NoClassDefFoundError: com/google/api/client/auth/oauth2/Credential
23/04/04 10:19:39 WARN FileSystem: java.lang.ClassNotFoundException: com.google.api.client.auth.oauth2.Credential


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 10:19:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Loaded 1713 rows and 17 columns


In [65]:
''' DEPRECATED moved to dailyTopN build_weighted_portfolio function'''

_cols={
    "PRIMARYKEY":'mcap_past_pk',
    "NAMECOLUMN":'asset_name',
    "DATECOLUMN":'mcap_date',
    "NUMCOLUMN" :'log_ror',
    "MCAPCOLUMN":'mcap_value',
    "WEIGHTCOLUMN":'weights',
    "MCAPSOURCE":'source',
}
_l_exp_wts,_cols_dict=clsMPT.get_weighted_mpt(
    data=_portfolio,
    cols_dict=_cols,
    topN=3,
    size=5,
)
_l_exp_wts[:2]

[{'source': 'warehouse.mcap_past',
  'mcap_past_pk': [81767, 207106, 141497],
  'mcap_date': datetime.datetime(2022, 1, 25, 0, 0),
  'asset_name': ['okb', 'atom', 'cro'],
  'log_ror': [Decimal('0.7294887529623831'),
   Decimal('0.7216704954131129'),
   Decimal('0.7027959808088148')],
  'weights': [0.874157354172054, 0.10634707169825057, 0.019495574129695534],
  'mcap_value': [Decimal('5922009410.9717900000000000'),
   Decimal('10596853538.6012000000000000'),
   Decimal('9715560017.1923800000000000')]}]